# BLOQUE 1: Introducción y Configuración del Entorno

In [ ]:
# Importamos las librerias necesarias
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings

# Configuraciones globales
warnings.filterwarnings("ignore") # Ignorar warnings para una salida más limpia
sns.set(style="whitegrid") # Estilo por defecto para Seaborn
pd.set_option('display.max_columns', None) # Mostrar todas las columnas en DataFrames
pd.set_option('display.float_format', lambda x: '%.2f' % x) # Formato de números flotantes

print("Librerías cargadas y configuración inicial aplicada.")

# BLOQUE 2: Generación de Datos Simulados

In [ ]:
# Semilla para reproducibilidad (importante para que los resultados sean consistentes)
np.random.seed(42)

# 1. Tabla de clientes
n_clients = 500
clientes = pd.DataFrame({
    "cliente_id": range(1, n_clients + 1),
    "nombre": [f"Cliente_{i}" for i in range(1, n_clients + 1)],
    "edad": np.random.randint(18, 75, size=n_clients),
    "genero": np.random.choice(["Masculino", "Femenino", "Otro"], size=n_clients, p=[0.48, 0.48, 0.04]), # Ampliamos género
    "ciudad": np.random.choice(["Madrid", "Barcelona", "Valencia", "Sevilla", "Bilbao", "Zaragoza"], size=n_clients), # Más ciudades
    "fecha_registro": pd.to_datetime( (pd.Timestamp('2015-01-01') + pd.to_timedelta(np.random.randint(0, 365*8, size=n_clients), unit='D')).date ) # Fechas entre 2015 y 2022
})

# 2. Tabla de cuentas
# Aseguramos que cada cliente tenga al menos una cuenta, algunos pueden tener más
num_cuentas_por_cliente = np.random.choice([1, 2, 3], size=n_clients, p=[0.7, 0.2, 0.1])
cuenta_cliente_ids = np.repeat(clientes["cliente_id"], num_cuentas_por_cliente)
n_cuentas = len(cuenta_cliente_ids)

cuentas = pd.DataFrame({
    "cuenta_id": range(1001, 1001 + n_cuentas), # IDs de cuenta únicos
    "cliente_id": cuenta_cliente_ids,
    "tipo_cuenta": np.random.choice(["Corriente", "Ahorros", "Nómina", "Inversión"], size=n_cuentas, p=[0.4, 0.3, 0.15, 0.15]), # Más tipos de cuenta
    "saldo": np.maximum(0, np.round(np.random.normal(5000, 7000, size=n_cuentas), 2)), # Saldos >= 0
    "fecha_apertura": clientes.loc[clientes["cliente_id"].isin(cuenta_cliente_ids), "fecha_registro"].sample(n_cuentas, replace=True).values # Fecha apertura posterior o igual a registro
})
# Ajuste para que fecha_apertura sea igual o posterior a fecha_registro del cliente
# Esto es un poco más complejo debido a la estructura de datos, simplificaremos asumiendo que la fecha de apertura es aleatoria después del registro.
# Para una simulación más precisa, iteraríamos o usaríamos un merge. Por ahora, la simplificación anterior es aceptable.
# Corregimos para que la fecha de apertura sea aleatoria entre la fecha de registro del cliente y hoy.
fechas_apertura_cuentas = []
for cid in cuentas['cliente_id']:
    fecha_reg_cliente = clientes[clientes['cliente_id'] == cid]['fecha_registro'].iloc[0]
    # Fecha de apertura entre fecha_registro y (fecha_registro + 3 años) o hasta hoy si es menor
    dias_desde_registro = (pd.Timestamp('now') - fecha_reg_cliente).days
    max_dias_apertura = min(365*3, dias_desde_registro if dias_desde_registro > 0 else 365*3) # Asegurar que no sea negativo
    dias_offset = np.random.randint(0, max_dias_apertura + 1) if max_dias_apertura >=0 else 0
    fechas_apertura_cuentas.append( (fecha_reg_cliente + pd.to_timedelta(dias_offset, unit='D')).date() )
cuentas['fecha_apertura'] = pd.to_datetime(fechas_apertura_cuentas)


# 3. Tabla de transacciones
n_transacciones = 10000 # Más transacciones para un análisis más rico
transacciones = pd.DataFrame({
    "transaccion_id": range(1, n_transacciones + 1),
    "cuenta_id": np.random.choice(cuentas["cuenta_id"], size=n_transacciones),
    "fecha_transaccion": pd.to_datetime( (pd.Timestamp('2020-01-01') + pd.to_timedelta(np.random.randint(0, 365*4, size=n_transacciones), unit='D')).date ), # Transacciones en los últimos 4 años
    "monto": np.round(np.random.lognormal(mean=np.log(100), sigma=1.5, size=n_transacciones), 2), # Montos con distribución log-normal (más realista)
    "tipo_transaccion": np.random.choice(["Ingreso", "Gasto", "Transferencia Saliente", "Transferencia Entrante", "Pago Servicio"], size=n_transacciones, p=[0.25, 0.35, 0.15, 0.15, 0.10]) # Más tipos de transacción
})

# Ajustar signos de montos para gastos y transferencias salientes
transacciones.loc[transacciones["tipo_transaccion"].isin(["Gasto", "Transferencia Saliente"]), "monto"] *= -1

# 4. Tabla de créditos
n_creditos = int(n_clients * 0.4) # 40% de los clientes tienen algún crédito
creditos = pd.DataFrame({
    "credito_id": range(1, n_creditos + 1),
    "cliente_id": np.random.choice(clientes["cliente_id"], size=n_creditos, replace=False),
    "monto_credito": np.random.choice([500, 1000, 5000, 10000, 20000, 50000, 100000, 250000], size=n_creditos, p=[0.1,0.15,0.2,0.2,0.15,0.1,0.05,0.05]), # Montos más variados
    "tasa_interes_anual": np.round(np.random.uniform(3.5, 15.0, size=n_creditos), 2), # Tasas de interés más realistas
    "estado_credito": np.random.choice(["Activo", "Pagado", "Incobrable", "Reestructurado"], size=n_creditos, p=[0.55, 0.25, 0.10, 0.10]), # Más estados
    "fecha_otorgamiento": pd.to_datetime( (pd.Timestamp('2018-01-01') + pd.to_timedelta(np.random.randint(0, 365*5, size=n_creditos), unit='D')).date ), # Créditos otorgados en los últimos 5 años
    "plazo_meses": np.random.choice([6, 12, 24, 36, 48, 60, 120, 240, 360], size=n_creditos) # Plazos comunes
})
# Asegurar que fecha_otorgamiento es posterior a fecha_registro del cliente
for index, row in creditos.iterrows():
    fecha_reg_cliente = clientes[clientes['cliente_id'] == row['cliente_id']]['fecha_registro'].iloc[0]
    if row['fecha_otorgamiento'] < fecha_reg_cliente:
        # Mover fecha_otorgamiento a después de fecha_registro + un pequeño delta aleatorio
        dias_offset_credito = np.random.randint(30, 365) # Otorgar crédito al menos 30 días después del registro
        creditos.loc[index, 'fecha_otorgamiento'] = (fecha_reg_cliente + pd.to_timedelta(dias_offset_credito, unit='D')).date()
creditos['fecha_otorgamiento'] = pd.to_datetime(creditos['fecha_otorgamiento'])


# Mostrar ejemplos y formas de los DataFrames
print("--- Clientes ---")
print(clientes.info())
print(clientes.head(3))
print("\n--- Cuentas ---")
print(cuentas.info())
print(cuentas.head(3))
print("\n--- Transacciones ---")
print(transacciones.info())
print(transacciones.head(3))
print("\n--- Créditos ---")
print(creditos.info())
print(creditos.head(3))

# BLOQUE 3: Crear Base de Datos SQLite e Insertar Datos

In [ ]:
# Crear una base de datos SQLite en disco (o en memoria con ":memory:")
db_name = "banco_analisis_v2.db"
conexion = sqlite3.connect(db_name)
cursor = conexion.cursor()

# Guardar cada DataFrame como tabla en SQL
# if_exists="replace" asegura que si corremos el notebook de nuevo, las tablas se sobrescriban
clientes.to_sql("clientes", conexion, if_exists="replace", index=False)
cuentas.to_sql("cuentas", conexion, if_exists="replace", index=False)
transacciones.to_sql("transacciones", conexion, if_exists="replace", index=False)
creditos.to_sql("creditos", conexion, if_exists="replace", index=False)

print(f"Base de datos '{db_name}' creada y tablas cargadas exitosamente.")

# Verificamos que las tablas fueron creadas correctamente y listamos sus columnas
print("\nTablas creadas en la base de datos:")
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablas = cursor.fetchall()
print(tablas)

for tabla_nombre_tupla in tablas:
    tabla_nombre = tabla_nombre_tupla[0]
    if tabla_nombre == 'sqlite_sequence': # Ignorar tabla interna de SQLite
        continue
    print(f"\nColumnas en la tabla '{tabla_nombre}':")
    cursor.execute(f"PRAGMA table_info('{tabla_nombre}');")
    columnas = cursor.fetchall()
    for col in columnas:
        print(f"  - {col[1]} ({col[2]})") # Nombre y tipo de dato

# BLOQUE 4: Análisis Exploratorio de Datos (EDA) con SQL y Python

In [ ]:
# Re-establecer conexión si es necesario (por ejemplo, si se reinició el kernel)
db_name = "banco_analisis_v2.db"
try:
    conexion.execute("SELECT 1") # Intenta ejecutar una consulta simple
    print("Conexión a la base de datos ya activa.")
except (sqlite3.ProgrammingError, NameError, AttributeError):
    print("Reconectando a la base de datos...")
    conexion = sqlite3.connect(db_name)
    cursor = conexion.cursor()
    print("Conexión reestablecida.")

# Función auxiliar para ejecutar y mostrar consultas SQL como DataFrames
def ejecutar_consulta_sql(sql_query, conn=conexion):
    """Ejecuta una consulta SQL y devuelve el resultado como un DataFrame de Pandas."""
    return pd.read_sql_query(sql_query, conn)

#### 4.1. Resumen General de las Entidades

Comencemos con algunas preguntas básicas para entender la escala de nuestros datos.

In [ ]:
# ¿Cuántos clientes hay en total?
consulta_total_clientes = "SELECT COUNT(*) AS total_clientes FROM clientes;"
df_total_clientes = ejecutar_consulta_sql(consulta_total_clientes)
print(f"Total de clientes en el banco: {df_total_clientes['total_clientes'].iloc[0]}")

# ¿Cuántas cuentas hay en total y por tipo?
consulta_total_cuentas = """
SELECT 
    tipo_cuenta,
    COUNT(*) AS numero_de_cuentas,
    ROUND(AVG(saldo), 2) AS saldo_promedio,
    ROUND(SUM(saldo), 2) AS saldo_total
FROM cuentas
GROUP BY tipo_cuenta
ORDER BY numero_de_cuentas DESC;
"""
df_total_cuentas = ejecutar_consulta_sql(consulta_total_cuentas)
print("\nResumen de Cuentas:")
print(df_total_cuentas)

# Visualización de tipos de cuenta
fig_tipos_cuenta = px.bar(df_total_cuentas, 
                          x='tipo_cuenta', 
                          y='numero_de_cuentas', 
                          color='tipo_cuenta',
                          text_auto=True,
                          title='Distribución de Cuentas por Tipo',
                          labels={'numero_de_cuentas': 'Número de Cuentas', 'tipo_cuenta': 'Tipo de Cuenta'})
fig_tipos_cuenta.update_layout(showlegend=False)
fig_tipos_cuenta.show()

#### 4.2. Perfil Demográfico de los Clientes

Entender la composición de nuestra base de clientes es clave para estrategias de marketing y producto.

In [ ]:
# Distribución de clientes por género
consulta_genero = """
SELECT 
    genero, 
    COUNT(*) AS numero_clientes
FROM clientes
GROUP BY genero
ORDER BY numero_clientes DESC;
"""
df_genero = ejecutar_consulta_sql(consulta_genero)
print("\nDistribución de Clientes por Género:")
print(df_genero)

fig_genero = px.pie(df_genero, 
                    names='genero', 
                    values='numero_clientes', 
                    title='Distribución de Clientes por Género',
                    hole=0.3)
fig_genero.show()

# Distribución de clientes por ciudad
consulta_ciudad = """
SELECT 
    ciudad, 
    COUNT(*) AS numero_clientes
FROM clientes
GROUP BY ciudad
ORDER BY numero_clientes DESC;
"""
df_ciudad = ejecutar_consulta_sql(consulta_ciudad)
print("\nDistribución de Clientes por Ciudad:")
print(df_ciudad)

fig_ciudad = px.bar(df_ciudad, 
                    x='ciudad', 
                    y='numero_clientes', 
                    color='ciudad',
                    text_auto=True,
                    title='Distribución de Clientes por Ciudad',
                    labels={'numero_clientes': 'Número de Clientes', 'ciudad': 'Ciudad'})
fig_ciudad.update_layout(xaxis_tickangle=-45)
fig_ciudad.show()

# Distribución de edades de los clientes
consulta_edades = "SELECT edad FROM clientes;"
df_edades = ejecutar_consulta_sql(consulta_edades)

plt.figure(figsize=(12, 6))
sns.histplot(df_edades['edad'], bins=30, kde=True, color="skyblue")
plt.title('Distribución de Edades de los Clientes', fontsize=15)
plt.xlabel('Edad', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.show()

print(f"\nEstadísticas descriptivas de la edad de los clientes:\n{df_edades['edad'].describe()}")

#### 4.3. Análisis Básico de Saldos y Transacciones

In [ ]:
# Distribución de saldos de cuentas (ya lo tenías, lo refinamos un poco)
consulta_saldos = "SELECT saldo FROM cuentas WHERE saldo > 0;" # Excluir saldos cero si no son relevantes
df_saldos_cuentas = ejecutar_consulta_sql(consulta_saldos)

plt.figure(figsize=(12, 6))
sns.histplot(df_saldos_cuentas['saldo'], bins=50, kde=True, color="teal", log_scale=False) # log_scale puede ser útil si hay mucha asimetría
plt.title('Distribución de Saldos de Cuentas', fontsize=15)
plt.xlabel('Saldo (€)', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
# plt.xscale('log') # Descomentar si la distribución es muy sesgada a la derecha
plt.show()

print(f"\nEstadísticas descriptivas de los saldos de las cuentas:\n{df_saldos_cuentas['saldo'].describe()}")
# Observar la diferencia entre media y mediana puede indicar asimetría.

# Resumen de transacciones por tipo
consulta_tipo_transaccion = """
SELECT 
    tipo_transaccion, 
    COUNT(*) AS cantidad, 
    ROUND(SUM(monto), 2) AS total_monto_neto,
    ROUND(AVG(ABS(monto)), 2) AS promedio_monto_absoluto
FROM transacciones
GROUP BY tipo_transaccion
ORDER BY cantidad DESC;
"""
df_tipo_transaccion = ejecutar_consulta_sql(consulta_tipo_transaccion)
print("\nResumen de Transacciones por Tipo:")
print(df_tipo_transaccion)

fig_trans_cantidad = px.bar(df_tipo_transaccion.sort_values('cantidad', ascending=False),
                           x='tipo_transaccion',
                           y='cantidad',
                           color='tipo_transaccion',
                           title='Número de Transacciones por Tipo',
                           text_auto=True)
fig_trans_cantidad.update_layout(xaxis_tickangle=-45)
fig_trans_cantidad.show()

fig_trans_monto = px.bar(df_tipo_transaccion.sort_values('total_monto_neto', ascending=False),
                         x='tipo_transaccion',
                         y='total_monto_neto',
                         color='tipo_transaccion',
                         title='Monto Neto Total Transaccionado por Tipo (€)',
                         text_auto=True)
fig_trans_monto.update_layout(xaxis_tickangle=-45)
fig_trans_monto.show()

#### 4.4. Análisis Básico de la Cartera de Créditos

In [ ]:
# Resumen de créditos por estado
consulta_estado_credito = """
SELECT 
    estado_credito, 
    COUNT(*) AS cantidad_creditos, 
    ROUND(AVG(monto_credito), 2) AS promedio_monto_credito,
    ROUND(SUM(monto_credito), 2) AS total_monto_credito,
    ROUND(AVG(tasa_interes_anual), 2) AS promedio_tasa_interes
FROM creditos
GROUP BY estado_credito
ORDER BY cantidad_creditos DESC;
"""
df_estado_credito = ejecutar_consulta_sql(consulta_estado_credito)
print("\nResumen de Créditos por Estado:")
print(df_estado_credito)

fig_cred_estado_cantidad = px.pie(df_estado_credito,
                                  names='estado_credito',
                                  values='cantidad_creditos',
                                  title='Distribución de Créditos por Estado (Cantidad)',
                                  hole=0.4)
fig_cred_estado_cantidad.show()

fig_cred_estado_monto = px.bar(df_estado_credito.sort_values('total_monto_credito', ascending=False),
                               x='estado_credito',
                               y='total_monto_credito',
                               color='estado_credito',
                               title='Monto Total de Créditos por Estado (€)',
                               text_auto=True)
fig_cred_estado_monto.show()

# Distribución de montos de crédito
consulta_montos_credito = "SELECT monto_credito FROM creditos;"
df_montos_credito = ejecutar_consulta_sql(consulta_montos_credito)

plt.figure(figsize=(12, 6))
sns.histplot(df_montos_credito['monto_credito'], bins=30, kde=True, color="purple")
plt.title('Distribución de Montos de Crédito Otorgados', fontsize=15)
plt.xlabel('Monto del Crédito (€)', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.show()
print(f"\nEstadísticas descriptivas de los montos de crédito:\n{df_montos_credito['monto_credito'].describe()}")

# BLOQUE 5: Consultas SQL Avanzadas para Insights de Negocio

#### 5.1. Segmentación de Clientes Avanzada

##### 5.1.1. Clientes por Valor (Saldo Total en Cuentas)

Identificar a los clientes con mayores saldos es fundamental para estrategias de retención y servicios premium.

In [ ]:
consulta_clientes_por_valor = """
WITH SaldosPorCliente AS (
    -- Calcula el saldo total para cada cliente sumando los saldos de todas sus cuentas
    SELECT
        cliente_id,
        SUM(saldo) AS saldo_total_cliente
    FROM cuentas
    GROUP BY cliente_id
),
RankingClientes AS (
    -- Asigna un ranking a cada cliente basado en su saldo total
    SELECT
        cliente_id,
        saldo_total_cliente,
        NTILE(4) OVER (ORDER BY saldo_total_cliente DESC) AS cuartil_saldo, -- Divide en 4 grupos (cuartiles)
        NTILE(10) OVER (ORDER BY saldo_total_cliente DESC) AS decil_saldo   -- Divide en 10 grupos (deciles)
    FROM SaldosPorCliente
)
-- Une la información del ranking con los detalles del cliente
SELECT
    c.cliente_id,
    c.nombre,
    c.edad,
    c.ciudad,
    rc.saldo_total_cliente,
    CASE rc.cuartil_saldo
        WHEN 1 THEN 'Top 25% (Valor Alto)'
        WHEN 2 THEN '25-50% (Valor Medio-Alto)'
        WHEN 3 THEN '50-75% (Valor Medio-Bajo)'
        ELSE 'Bottom 25% (Valor Bajo)'
    END AS segmento_valor_cuartil,
    rc.decil_saldo
FROM clientes c
JOIN RankingClientes rc ON c.cliente_id = rc.cliente_id
ORDER BY rc.saldo_total_cliente DESC;
"""

df_clientes_valor = ejecutar_consulta_sql(consulta_clientes_por_valor)
print("\nTop 10 Clientes por Saldo Total:")
print(df_clientes_valor.head(10))

print(f"\nTotal de clientes segmentados: {len(df_clientes_valor)}")

# Resumen de la segmentación por cuartiles
print("\nResumen de Segmentación por Cuartil de Saldo:")
segmentacion_resumen = df_clientes_valor.groupby('segmento_valor_cuartil')['saldo_total_cliente'].agg(['count', 'mean', 'sum'])
segmentacion_resumen.columns = ['Numero Clientes', 'Saldo Promedio', 'Saldo Total Acumulado']
print(segmentacion_resumen.sort_values('Saldo Promedio', ascending=False))

# Visualización de la distribución de saldos por segmento de valor
fig_valor_segmento = px.box(df_clientes_valor, 
                            x='segmento_valor_cuartil', 
                            y='saldo_total_cliente', 
                            color='segmento_valor_cuartil',
                            title='Distribución de Saldos Totales por Segmento de Valor (Cuartiles)',
                            labels={'saldo_total_cliente': 'Saldo Total del Cliente (€)', 'segmento_valor_cuartil': 'Segmento de Valor'},
                            category_orders={"segmento_valor_cuartil": ['Top 25% (Valor Alto)', '25-50% (Valor Medio-Alto)', '50-75% (Valor Medio-Bajo)', 'Bottom 25% (Valor Bajo)']})
fig_valor_segmento.show()

##### 5.1.2. Antigüedad del Cliente y su Relación con el Saldo y Créditos

¿Los clientes más antiguos tienden a tener mayores saldos o más créditos?
SQLite no tiene una función `DATEDIFF` directa como SQL Server o MySQL para calcular diferencias de fechas fácilmente en días o años. Usaremos `julianday`.
La función `strftime('%Y', 'now') - strftime('%Y', fecha_registro)` es una aproximación para años, pero `julianday` es más preciso para diferencias.

In [ ]:
consulta_antiguedad_valor = """
SELECT
    c.cliente_id,
    c.nombre,
    c.fecha_registro,
    (julianday('now') - julianday(c.fecha_registro)) / 365.25 AS antiguedad_anos, -- Antigüedad en años
    COALESCE(SUM(cu.saldo), 0) AS saldo_total_cliente,
    COUNT(DISTINCT cr.credito_id) AS numero_creditos_activos -- Contamos solo créditos activos
FROM clientes c
LEFT JOIN cuentas cu ON c.cliente_id = cu.cliente_id
LEFT JOIN creditos cr ON c.cliente_id = cr.cliente_id AND cr.estado_credito = 'Activo'
GROUP BY c.cliente_id, c.nombre, c.fecha_registro
ORDER BY antiguedad_anos DESC;
"""
df_antiguedad_valor = ejecutar_consulta_sql(consulta_antiguedad_valor)
# Convertir antigüedad a numérico, ya que julianday puede devolver strings en algunos contextos de pandas.
df_antiguedad_valor['antiguedad_anos'] = pd.to_numeric(df_antiguedad_valor['antiguedad_anos'], errors='coerce')


print("\nClientes por Antigüedad, Saldo Total y Número de Créditos Activos:")
print(df_antiguedad_valor.head())

# Crear categorías de antigüedad para un mejor análisis agrupado
bins_antiguedad = [0, 2, 5, 8, df_antiguedad_valor['antiguedad_anos'].max() + 1] # Ej: 0-2 años, 2-5 años, 5-8 años, 8+ años
labels_antiguedad = ['0-2 años', '2-5 años', '5-8 años', '8+ años']
df_antiguedad_valor['grupo_antiguedad'] = pd.cut(df_antiguedad_valor['antiguedad_anos'], bins=bins_antiguedad, labels=labels_antiguedad, right=False)

resumen_antiguedad = df_antiguedad_valor.groupby('grupo_antiguedad').agg(
    saldo_promedio=('saldo_total_cliente', 'mean'),
    creditos_promedio=('numero_creditos_activos', 'mean'),
    numero_clientes=('cliente_id', 'count')
).reset_index()

print("\nResumen por Grupo de Antigüedad:")
print(resumen_antiguedad)

fig_ant_saldo = px.bar(resumen_antiguedad, 
                       x='grupo_antiguedad', 
                       y='saldo_promedio', 
                       title='Saldo Promedio por Grupo de Antigüedad del Cliente',
                       text_auto='.2s',
                       labels={'saldo_promedio': 'Saldo Promedio (€)', 'grupo_antiguedad': 'Grupo de Antigüedad'})
fig_ant_saldo.show()

fig_ant_creditos = px.bar(resumen_antiguedad, 
                          x='grupo_antiguedad', 
                          y='creditos_promedio', 
                          title='Número Promedio de Créditos Activos por Grupo de Antigüedad',
                          text_auto='.2f',
                          labels={'creditos_promedio': 'Promedio de Créditos Activos', 'grupo_antiguedad': 'Grupo de Antigüedad'})
fig_ant_creditos.show()

# Scatter plot para ver la relación individual (ya lo tenías, lo mantenemos y mejoramos)
fig_scatter_ant_saldo = px.scatter(df_antiguedad_valor, 
                                   x="antiguedad_anos", 
                                   y="saldo_total_cliente", 
                                   trendline="ols", # Ordinary Least Squares regression line
                                   title="Relación entre Antigüedad del Cliente y Saldo Total",
                                   labels={'antiguedad_anos': 'Antigüedad (Años)', 'saldo_total_cliente': 'Saldo Total (€)'},
                                   hover_data=['nombre'])
fig_scatter_ant_saldo.show()

#### 5.2. Análisis Profundo de la Cartera de Créditos

##### 5.2.1. Morosidad por Perfil de Cliente (Edad y Ciudad)

Identificar qué segmentos de clientes presentan mayor riesgo de incumplimiento.

In [ ]:
consulta_morosidad_perfil = """
WITH CreditosCliente AS (
    -- Unimos créditos con información del cliente
    SELECT
        cr.credito_id,
        cr.cliente_id,
        cl.edad,
        cl.ciudad,
        cr.monto_credito,
        cr.estado_credito,
        CASE
            WHEN cl.edad < 25 THEN 'Menos de 25'
            WHEN cl.edad BETWEEN 25 AND 34 THEN '25-34'
            WHEN cl.edad BETWEEN 35 AND 44 THEN '35-44'
            WHEN cl.edad BETWEEN 45 AND 54 THEN '45-54'
            WHEN cl.edad BETWEEN 55 AND 64 THEN '55-64'
            ELSE '65+'
        END AS grupo_edad
    FROM creditos cr
    JOIN clientes cl ON cr.cliente_id = cl.cliente_id
)
-- Calculamos la tasa de morosidad (créditos incobrables / total de créditos)
SELECT
    grupo_edad,
    ciudad,
    COUNT(*) AS total_creditos,
    SUM(CASE WHEN estado_credito = 'Incobrable' THEN 1 ELSE 0 END) AS creditos_incobrables,
    ROUND(
        SUM(CASE WHEN estado_credito = 'Incobrable' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2
    ) AS tasa_morosidad_pct,
    ROUND(AVG(monto_credito),2) AS monto_promedio_credito
FROM CreditosCliente
GROUP BY grupo_edad, ciudad
ORDER BY tasa_morosidad_pct DESC, total_creditos DESC;
"""

df_morosidad_perfil = ejecutar_consulta_sql(consulta_morosidad_perfil)
print("\nTasa de Morosidad por Grupo de Edad y Ciudad:")
print(df_morosidad_perfil.head(15)) # Mostrar los más relevantes

# Visualización: Heatmap de tasa de morosidad
# Para el heatmap, necesitamos pivotar la tabla.
try:
    heatmap_data_morosidad = df_morosidad_perfil.pivot_table(
        index='grupo_edad', 
        columns='ciudad', 
        values='tasa_morosidad_pct',
        fill_value=0 # Llenar NaN con 0 para ciudades/grupos sin créditos o sin morosidad
    )
    # Ordenar el índice (grupos de edad)
    edad_order = ['Menos de 25', '25-34', '35-44', '45-54', '55-64', '65+']
    heatmap_data_morosidad = heatmap_data_morosidad.reindex(edad_order)


    plt.figure(figsize=(12, 8))
    sns.heatmap(heatmap_data_morosidad, annot=True, fmt=".1f", cmap="Reds", linewidths=.5)
    plt.title('Heatmap de Tasa de Morosidad (%) por Grupo de Edad y Ciudad', fontsize=15)
    plt.xlabel('Ciudad', fontsize=12)
    plt.ylabel('Grupo de Edad', fontsize=12)
    plt.show()
except KeyError as e:
    print(f"Error al generar el heatmap, posible falta de datos para pivotar: {e}")
    print("Mostrando datos tabulares en su lugar si el heatmap falla.")
    print(df_morosidad_perfil)


# También podemos ver la morosidad general por grupo de edad
consulta_morosidad_edad = """
WITH CreditosCliente AS (
    SELECT
        cr.cliente_id,
        cl.edad,
        cr.estado_credito,
        CASE
            WHEN cl.edad < 25 THEN 'Menos de 25'
            WHEN cl.edad BETWEEN 25 AND 34 THEN '25-34'
            WHEN cl.edad BETWEEN 35 AND 44 THEN '35-44'
            WHEN cl.edad BETWEEN 45 AND 54 THEN '45-54'
            WHEN cl.edad BETWEEN 55 AND 64 THEN '55-64'
            ELSE '65+'
        END AS grupo_edad
    FROM creditos cr
    JOIN clientes cl ON cr.cliente_id = cl.cliente_id
)
SELECT
    grupo_edad,
    COUNT(*) AS total_creditos,
    SUM(CASE WHEN estado_credito = 'Incobrable' THEN 1 ELSE 0 END) AS creditos_incobrables,
    ROUND(
        SUM(CASE WHEN estado_credito = 'Incobrable' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2
    ) AS tasa_morosidad_pct
FROM CreditosCliente
GROUP BY grupo_edad
ORDER BY grupo_edad;
"""
df_morosidad_edad = ejecutar_consulta_sql(consulta_morosidad_edad)
fig_morosidad_edad = px.bar(df_morosidad_edad, 
                            x='grupo_edad', 
                            y='tasa_morosidad_pct', 
                            text='tasa_morosidad_pct',
                            title='Tasa de Morosidad (%) por Grupo de Edad',
                            labels={'tasa_morosidad_pct': 'Tasa de Morosidad (%)', 'grupo_edad': 'Grupo de Edad'})
fig_morosidad_edad.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig_morosidad_edad.show()

##### 5.2.2. Análisis Vintage de Créditos (Año de Otorgamiento vs. Estado Actual)

El análisis vintage ayuda a entender si la calidad de los créditos otorgados ha mejorado o empeorado con el tiempo. Compara cohortes de créditos basadas en su fecha de otorgamiento.

In [ ]:
consulta_vintage_creditos = """
SELECT
    strftime('%Y', fecha_otorgamiento) AS anio_otorgamiento,
    estado_credito,
    COUNT(*) AS numero_de_creditos,
    ROUND(SUM(monto_credito), 2) AS monto_total_creditos
FROM creditos
GROUP BY anio_otorgamiento, estado_credito
ORDER BY anio_otorgamiento, estado_credito;
"""
df_vintage_creditos = ejecutar_consulta_sql(consulta_vintage_creditos)
print("\nAnálisis Vintage de Créditos (Cantidad y Monto por Año de Otorgamiento y Estado):")
print(df_vintage_creditos)

# Calcular el porcentaje de cada estado_credito dentro de cada anio_otorgamiento
df_vintage_pivot = df_vintage_creditos.pivot_table(
    index='anio_otorgamiento',
    columns='estado_credito',
    values='numero_de_creditos',
    fill_value=0
)
df_vintage_porcentaje = df_vintage_pivot.apply(lambda x: x * 100 / x.sum(), axis=1)
df_vintage_porcentaje = df_vintage_porcentaje.reset_index()
print("\nAnálisis Vintage de Créditos (Porcentaje por Estado dentro de cada Año):")
print(df_vintage_porcentaje)


# Visualización: Gráfico de barras apiladas por porcentaje
# Necesitamos transformar los datos a un formato 'long' para Plotly Express
df_vintage_plot = df_vintage_porcentaje.melt(
    id_vars='anio_otorgamiento',
    var_name='estado_credito',
    value_name='porcentaje_creditos'
)

fig_vintage = px.bar(df_vintage_plot,
                     x='anio_otorgamiento',
                     y='porcentaje_creditos',
                     color='estado_credito',
                     title='Análisis Vintage: Distribución Porcentual del Estado de Créditos por Año de Otorgamiento',
                     labels={'anio_otorgamiento': 'Año de Otorgamiento', 'porcentaje_creditos': '% del Total de Créditos del Año'},
                     barmode='stack', # o 'group' para barras agrupadas
                     text_auto='.1f')
fig_vintage.update_layout(yaxis_ticksuffix='%')
fig_vintage.show()

#### 5.3. Análisis de Transaccionalidad y Comportamiento

##### 5.3.1. Clientes por Frecuencia y Monto Promedio de Transacciones

Segmentar clientes según su actividad transaccional.

In [ ]:
consulta_actividad_transaccional = """
WITH TransaccionesCliente AS (
    -- Agrega transacciones a nivel de cliente
    SELECT
        cu.cliente_id,
        t.transaccion_id,
        t.monto,
        ABS(t.monto) AS monto_absoluto -- Para el promedio, el signo no importa
    FROM transacciones t
    JOIN cuentas cu ON t.cuenta_id = cu.cuenta_id
),
AgregadosCliente AS (
    -- Calcula métricas de actividad por cliente
    SELECT
        cliente_id,
        COUNT(transaccion_id) AS numero_transacciones,
        AVG(monto_absoluto) AS monto_promedio_transaccion,
        SUM(CASE WHEN monto > 0 THEN monto ELSE 0 END) AS total_ingresos_cliente,
        SUM(CASE WHEN monto < 0 THEN monto ELSE 0 END) AS total_gastos_cliente -- Será negativo
    FROM TransaccionesCliente
    GROUP BY cliente_id
)
-- Une con la tabla de clientes para obtener más detalles
SELECT
    cl.cliente_id,
    cl.nombre,
    ac.numero_transacciones,
    ROUND(ac.monto_promedio_transaccion, 2) AS monto_promedio_transaccion,
    ROUND(ac.total_ingresos_cliente, 2) AS total_ingresos_cliente,
    ROUND(ABS(ac.total_gastos_cliente), 2) AS total_gastos_cliente -- Mostrar como positivo
FROM clientes cl
JOIN AgregadosCliente ac ON cl.cliente_id = ac.cliente_id
ORDER BY ac.numero_transacciones DESC, ac.monto_promedio_transaccion DESC;
"""

df_actividad_transaccional = ejecutar_consulta_sql(consulta_actividad_transaccional)
print("\nClientes por Actividad Transaccional (Frecuencia y Monto Promedio):")
print(df_actividad_transaccional.head(10))

# Scatter plot para visualizar la relación entre frecuencia y monto promedio
fig_actividad_scatter = px.scatter(df_actividad_transaccional,
                                   x='numero_transacciones',
                                   y='monto_promedio_transaccion',
                                   size='total_ingresos_cliente', # El tamaño de la burbuja puede representar otra métrica
                                   color='total_ingresos_cliente', # Colorear por ingresos
                                   hover_name='nombre',
                                   title='Actividad Transaccional de Clientes: Frecuencia vs. Monto Promedio',
                                   labels={
                                       'numero_transacciones': 'Número de Transacciones',
                                       'monto_promedio_transaccion': 'Monto Promedio por Transacción (€)',
                                       'total_ingresos_cliente': 'Ingresos Totales del Cliente (€)'
                                   },
                                   size_max=30)
fig_actividad_scatter.show()

##### 5.3.2. Patrones de Gasto Mensual por Cliente (Top N Categorías - Simulado)

Aunque no tenemos categorías de gasto explícitas en la tabla `transacciones` (como "Supermercado", "Restaurantes", etc.), podemos simular este análisis agrupando por `tipo_transaccion` y enfocándonos en los gastos. En un escenario real, tendríamos datos de categorización de comercios.
Por ahora, analizaremos la distribución de los gastos entre los diferentes `tipo_transaccion` que implican una salida de dinero.

In [ ]:
consulta_patrones_gasto = """
WITH GastosCliente AS (
    -- Selecciona solo las transacciones de gasto y calcula el mes
    SELECT
        cu.cliente_id,
        strftime('%Y-%m', t.fecha_transaccion) AS anio_mes,
        t.tipo_transaccion,
        ABS(t.monto) AS monto_gasto -- Monto en positivo para el análisis
    FROM transacciones t
    JOIN cuentas cu ON t.cuenta_id = cu.cuenta_id
    WHERE t.monto < 0 -- Solo transacciones de salida (gastos, transferencias salientes)
),
ResumenGastoMensual AS (
    -- Agrupa los gastos por cliente, mes y tipo de transacción
    SELECT
        cliente_id,
        anio_mes,
        tipo_transaccion,
        SUM(monto_gasto) AS total_gastado_tipo
    FROM GastosCliente
    GROUP BY cliente_id, anio_mes, tipo_transaccion
),
RankingGastoMensual AS (
    -- Rankea los tipos de transacción por gasto total para cada cliente y mes
    SELECT
        cliente_id,
        anio_mes,
        tipo_transaccion,
        total_gastado_tipo,
        RANK() OVER (PARTITION BY cliente_id, anio_mes ORDER BY total_gastado_tipo DESC) AS ranking_gasto
    FROM ResumenGastoMensual
)
-- Selecciona los principales tipos de gasto por cliente y mes (ej. Top 3)
SELECT
    rgm.cliente_id,
    cl.nombre AS nombre_cliente,
    rgm.anio_mes,
    rgm.tipo_transaccion,
    ROUND(rgm.total_gastado_tipo, 2) AS total_gastado_tipo,
    rgm.ranking_gasto
FROM RankingGastoMensual rgm
JOIN clientes cl ON rgm.cliente_id = cl.cliente_id
WHERE rgm.ranking_gasto <= 3 -- Mostrar los 3 principales tipos de gasto
ORDER BY rgm.cliente_id, rgm.anio_mes, rgm.ranking_gasto
LIMIT 30; -- Mostrar solo una muestra para no saturar la salida
"""
df_patrones_gasto = ejecutar_consulta_sql(consulta_patrones_gasto)
print("\nPrincipales Tipos de Gasto Mensual por Cliente (Muestra):")
print(df_patrones_gasto)

# Para una visualización agregada, podríamos ver los tipos de gasto más comunes en general
consulta_gasto_agregado_tipo = """
SELECT
    tipo_transaccion,
    SUM(ABS(monto)) AS total_gastado_general,
    COUNT(*) AS numero_transacciones_gasto
FROM transacciones
WHERE monto < 0
GROUP BY tipo_transaccion
ORDER BY total_gastado_general DESC;
"""
df_gasto_agregado_tipo = ejecutar_consulta_sql(consulta_gasto_agregado_tipo)
print("\nTotal Gastado Generalmente por Tipo de Transacción de Salida:")
print(df_gasto_agregado_tipo)

fig_gasto_agregado = px.bar(df_gasto_agregado_tipo,
                           x='tipo_transaccion',
                           y='total_gastado_general',
                           color='tipo_transaccion',
                           title='Distribución General de Gastos por Tipo de Transacción',
                           text_auto='.2s',
                           labels={'total_gastado_general': 'Total Gastado (€)', 'tipo_transaccion': 'Tipo de Transacción de Gasto'})
fig_gasto_agregado.update_layout(xaxis_tickangle=-45)
fig_gasto_agregado.show()

#### 5.4. Indicadores Clave de Rendimiento (KPIs) Financieros Agregados

##### 5.4.1. Ingreso Neto Mensual y Acumulado (Considerando Transacciones)

Seguimiento de la rentabilidad operativa basada en flujos de transacciones.
(Nota: Esto es una simplificación. Un ingreso neto real consideraría intereses de créditos, comisiones, etc. Aquí nos basamos solo en la tabla `transacciones`).

In [ ]:
consulta_ingreso_neto_mensual = """
WITH FlujoMensual AS (
    -- Calcula ingresos y gastos totales por mes
    SELECT
        strftime('%Y-%m', fecha_transaccion) AS anio_mes,
        SUM(CASE WHEN tipo_transaccion IN ('Ingreso', 'Transferencia Entrante') THEN monto ELSE 0 END) AS total_ingresos_mes,
        SUM(CASE WHEN tipo_transaccion IN ('Gasto', 'Transferencia Saliente', 'Pago Servicio') THEN monto ELSE 0 END) AS total_gastos_mes -- Será negativo
    FROM transacciones
    GROUP BY anio_mes
),
NetoMensual AS (
    -- Calcula el ingreso neto mensual
    SELECT
        anio_mes,
        total_ingresos_mes,
        ABS(total_gastos_mes) AS total_gastos_mes_abs, -- Gastos en positivo para visualización
        (total_ingresos_mes + total_gastos_mes) AS ingreso_neto_mes -- gastos_mes es negativo, por eso se suma
    FROM FlujoMensual
)
-- Calcula el ingreso neto acumulado usando una Window Function
SELECT
    anio_mes,
    ROUND(total_ingresos_mes, 2) AS total_ingresos_mes,
    ROUND(total_gastos_mes_abs, 2) AS total_gastos_mes,
    ROUND(ingreso_neto_mes, 2) AS ingreso_neto_mes,
    ROUND(SUM(ingreso_neto_mes) OVER (ORDER BY anio_mes ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW), 2) AS ingreso_neto_acumulado
FROM NetoMensual
ORDER BY anio_mes;
"""
df_ingreso_neto_mensual = ejecutar_consulta_sql(consulta_ingreso_neto_mensual)
print("\nIngreso Neto Mensual y Acumulado (Basado en Transacciones):")
print(df_ingreso_neto_mensual)

# Visualización de evolución mensual y acumulada
fig_ing_neto_mensual = px.bar(df_ingreso_neto_mensual, 
                             x='anio_mes', 
                             y='ingreso_neto_mes',
                             title='Ingreso Neto Mensual (Transacciones)',
                             labels={'ingreso_neto_mes': 'Ingreso Neto (€)', 'anio_mes': 'Año-Mes'},
                             text_auto='.2s')
fig_ing_neto_mensual.add_scatter(x=df_ingreso_neto_mensual['anio_mes'], 
                                 y=df_ingreso_neto_mensual['ingreso_neto_acumulado'], 
                                 mode='lines+markers', 
                                 name='Ingreso Neto Acumulado',
                                 yaxis='y2') # Usar un segundo eje Y para la línea acumulada
fig_ing_neto_mensual.update_layout(
    yaxis_title='Ingreso Neto Mensual (€)',
    yaxis2=dict(
        title='Ingreso Neto Acumulado (€)',
        overlaying='y',
        side='right'
    ),
    legend_title_text='Métricas'
)
fig_ing_neto_mensual.show()

##### 5.4.2. Ratio de Calidad de Activos: Créditos Incobrables sobre Total de Créditos

Un indicador clave del riesgo de la cartera de créditos. (Similar a lo que tenías, pero reforzado).

In [ ]:
consulta_ratio_calidad_activos = """
SELECT
    'General' AS segmento, -- Para poder añadir más segmentos si fuera necesario
    COUNT(*) AS total_creditos,
    SUM(CASE WHEN estado_credito = 'Incobrable' THEN 1 ELSE 0 END) AS creditos_incobrables,
    SUM(CASE WHEN estado_credito = 'Incobrable' THEN monto_credito ELSE 0 END) AS monto_incobrable,
    SUM(monto_credito) AS monto_total_creditos,
    ROUND(
        SUM(CASE WHEN estado_credito = 'Incobrable' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2
    ) AS ratio_incobrabilidad_por_numero_pct, -- Tasa de morosidad por número de créditos
    ROUND(
        SUM(CASE WHEN estado_credito = 'Incobrable' THEN monto_credito ELSE 0 END) * 100.0 / SUM(monto_credito), 2
    ) AS ratio_incobrabilidad_por_monto_pct -- Tasa de morosidad por monto (más importante financieramente)
FROM creditos;
"""
df_ratio_calidad_activos = ejecutar_consulta_sql(consulta_ratio_calidad_activos)
print("\nRatio de Calidad de Activos (Cartera de Créditos):")
print(df_ratio_calidad_activos.T) # Transponer para mejor visualización de una sola fila

# Podríamos extender esto por año de otorgamiento para ver si la calidad cambia
consulta_ratio_calidad_vintage = """
SELECT
    strftime('%Y', fecha_otorgamiento) AS anio_otorgamiento,
    COUNT(*) AS total_creditos,
    SUM(CASE WHEN estado_credito = 'Incobrable' THEN 1 ELSE 0 END) AS creditos_incobrables,
    SUM(CASE WHEN estado_credito = 'Incobrable' THEN monto_credito ELSE 0 END) AS monto_incobrable,
    SUM(monto_credito) AS monto_total_creditos,
    ROUND(
        SUM(CASE WHEN estado_credito = 'Incobrable' THEN monto_credito ELSE 0 END) * 100.0 / SUM(monto_credito), 2
    ) AS ratio_incobrabilidad_por_monto_pct
FROM creditos
GROUP BY anio_otorgamiento
ORDER BY anio_otorgamiento;
"""
df_ratio_calidad_vintage = ejecutar_consulta_sql(consulta_ratio_calidad_vintage)
df_ratio_calidad_vintage.fillna(0, inplace=True) # En caso de años sin créditos o sin incobrables

print("\nRatio de Incobrabilidad por Monto (Vintage Analysis):")
print(df_ratio_calidad_vintage)

fig_ratio_vintage = px.line(df_ratio_calidad_vintage,
                            x='anio_otorgamiento',
                            y='ratio_incobrabilidad_por_monto_pct',
                            title='Evolución de la Tasa de Incobrabilidad de Créditos por Monto (Vintage)',
                            markers=True,
                            labels={'ratio_incobrabilidad_por_monto_pct': 'Tasa de Incobrabilidad (%)', 'anio_otorgamiento': 'Año de Otorgamiento'})
fig_ratio_vintage.update_layout(yaxis_ticksuffix='%')
fig_ratio_vintage.show()

# BLOQUE 6: Visualización Avanzada y Simulación de Dashboards (Estilo Power BI)

In [ ]:
# Función auxiliar para ejecutar y mostrar consultas SQL como DataFrames (ya definida)
# def ejecutar_consulta_sql(sql_query, conn=conexion):
#     return pd.read_sql_query(sql_query, conn)

# Recordar DataFrames clave que podríamos usar de bloques anteriores o recalcularlos si es necesario
# df_clientes_valor (Segmentación por valor)
# df_antiguedad_valor (Antigüedad vs Saldo/Créditos)
# df_morosidad_perfil (Morosidad por edad y ciudad)
# df_vintage_creditos (Análisis Vintage)
# df_actividad_transaccional (Actividad de clientes)
# df_ingreso_neto_mensual (KPI financiero)
# df_ratio_calidad_vintage (KPI de riesgo)

#### 6.1. Dashboard de Resumen del Cliente (Simulación)

Imaginemos un dashboard en Power BI donde, al seleccionar un cliente, se muestran sus KPIs principales. Aquí simularemos la obtención de datos para un cliente específico y algunas visualizaciones asociadas.

In [ ]:
# Seleccionar un cliente de ejemplo (podríamos tomar uno de alto valor)
cliente_ejemplo_id = df_clientes_valor.iloc[0]['cliente_id'] # El cliente con mayor saldo
nombre_cliente_ejemplo = df_clientes_valor.iloc[0]['nombre']

print(f"--- Simulación de Dashboard para el Cliente: {nombre_cliente_ejemplo} (ID: {cliente_ejemplo_id}) ---")

# 1. Información General y Saldos del Cliente
consulta_info_cliente = f"""
SELECT
    cl.cliente_id,
    cl.nombre,
    cl.edad,
    cl.genero,
    cl.ciudad,
    STRFTIME('%Y-%m-%d', cl.fecha_registro) AS fecha_registro,
    (julianday('now') - julianday(cl.fecha_registro)) / 365.25 AS antiguedad_anos,
    (SELECT COUNT(*) FROM cuentas cu WHERE cu.cliente_id = cl.cliente_id) AS numero_cuentas,
    (SELECT SUM(saldo) FROM cuentas cu WHERE cu.cliente_id = cl.cliente_id) AS saldo_total
FROM clientes cl
WHERE cl.cliente_id = {cliente_ejemplo_id};
"""
df_info_cliente_sel = ejecutar_consulta_sql(consulta_info_cliente)
print("\nInformación General del Cliente:")
print(df_info_cliente_sel.T)

# 2. Cuentas del Cliente
consulta_cuentas_cliente = f"""
SELECT
    cuenta_id,
    tipo_cuenta,
    ROUND(saldo, 2) AS saldo,
    STRFTIME('%Y-%m-%d', fecha_apertura) AS fecha_apertura
FROM cuentas
WHERE cliente_id = {cliente_ejemplo_id}
ORDER BY saldo DESC;
"""
df_cuentas_cliente_sel = ejecutar_consulta_sql(consulta_cuentas_cliente)
print("\nCuentas del Cliente:")
print(df_cuentas_cliente_sel)

if not df_cuentas_cliente_sel.empty:
    fig_cuentas_cliente = px.bar(df_cuentas_cliente_sel, 
                                 x='tipo_cuenta', 
                                 y='saldo', 
                                 color='tipo_cuenta',
                                 title=f'Saldos por Tipo de Cuenta para {nombre_cliente_ejemplo}',
                                 text_auto='.2s')
    fig_cuentas_cliente.show()

# 3. Créditos del Cliente
consulta_creditos_cliente = f"""
SELECT
    credito_id,
    ROUND(monto_credito, 2) AS monto_credito,
    tasa_interes_anual,
    estado_credito,
    STRFTIME('%Y-%m-%d', fecha_otorgamiento) AS fecha_otorgamiento,
    plazo_meses
FROM creditos
WHERE cliente_id = {cliente_ejemplo_id}
ORDER BY fecha_otorgamiento DESC;
"""
df_creditos_cliente_sel = ejecutar_consulta_sql(consulta_creditos_cliente)
print("\nCréditos del Cliente:")
if df_creditos_cliente_sel.empty:
    print("Este cliente no tiene créditos.")
else:
    print(df_creditos_cliente_sel)
    # En Power BI, esto sería una tabla o tarjetas individuales por crédito.

# 4. Actividad Transaccional Reciente del Cliente (Ej: últimos 6 meses, últimas 10 transacciones)
consulta_transacciones_cliente = f"""
SELECT
    t.transaccion_id,
    STRFTIME('%Y-%m-%d %H:%M', t.fecha_transaccion) AS fecha_transaccion, -- Asumimos que no tenemos hora, pero es buena práctica
    t.tipo_transaccion,
    ROUND(t.monto, 2) AS monto,
    cu.cuenta_id,
    cu.tipo_cuenta
FROM transacciones t
JOIN cuentas cu ON t.cuenta_id = cu.cuenta_id
WHERE cu.cliente_id = {cliente_ejemplo_id}
  -- AND t.fecha_transaccion >= date('now', '-6 months') -- Para últimos 6 meses (SQLite)
ORDER BY t.fecha_transaccion DESC
LIMIT 10;
"""
df_transacciones_cliente_sel = ejecutar_consulta_sql(consulta_transacciones_cliente)
print("\nÚltimas Transacciones del Cliente:")
if df_transacciones_cliente_sel.empty:
    print("Este cliente no tiene transacciones recientes o registradas con este criterio.")
else:
    print(df_transacciones_cliente_sel)
    
    # Visualización del flujo de caja mensual simplificado para este cliente
    consulta_flujo_cliente = f"""
    SELECT
        STRFTIME('%Y-%m', t.fecha_transaccion) AS anio_mes,
        SUM(CASE WHEN t.monto > 0 THEN t.monto ELSE 0 END) as ingresos_mes,
        ABS(SUM(CASE WHEN t.monto < 0 THEN t.monto ELSE 0 END)) as gastos_mes
    FROM transacciones t
    JOIN cuentas cu ON t.cuenta_id = cu.cuenta_id
    WHERE cu.cliente_id = {cliente_ejemplo_id}
    GROUP BY anio_mes
    ORDER BY anio_mes;
    """
    df_flujo_cliente_sel = ejecutar_consulta_sql(consulta_flujo_cliente)
    if not df_flujo_cliente_sel.empty:
        fig_flujo_cliente = px.bar(df_flujo_cliente_sel, 
                                     x='anio_mes', 
                                     y=['ingresos_mes', 'gastos_mes'],
                                     barmode='group',
                                     title=f'Ingresos vs. Gastos Mensuales para {nombre_cliente_ejemplo}',
                                     labels={'value': 'Monto (€)', 'anio_mes': 'Mes'})
        fig_flujo_cliente.show()

#### 6.2. Dashboard de Rendimiento de la Cartera de Créditos

Un dashboard enfocado en la salud y rentabilidad de la cartera de créditos.

In [ ]:
print(f"\n--- Simulación de Dashboard de Rendimiento de la Cartera de Créditos ---")

# 1. KPIs Principales de la Cartera (Ya calculados, los mostramos)
print("\nRatio de Calidad de Activos (Cartera de Créditos):")
print(df_ratio_calidad_activos.T) # Transpuesto para legibilidad

# 2. Distribución de Créditos por Estado y Monto (Ya generado en Bloque 4, lo recordamos)
# df_estado_credito (contiene cantidad, monto promedio, monto total por estado)
if 'df_estado_credito' in globals():
    fig_cred_estado_monto_total = px.bar(df_estado_credito.sort_values('total_monto_credito', ascending=False),
                                   x='estado_credito',
                                   y='total_monto_credito',
                                   color='estado_credito',
                                   title='Monto Total de Créditos por Estado (€)',
                                   text_auto='.2s')
    fig_cred_estado_monto_total.show()

    fig_cred_estado_cantidad = px.pie(df_estado_credito,
                                      names='estado_credito',
                                      values='cantidad_creditos',
                                      title='Distribución de Créditos por Estado (Cantidad)',
                                      hole=0.4,
                                      color_discrete_sequence=px.colors.sequential.RdBu)
    fig_cred_estado_cantidad.show()


# 3. Tasa de Morosidad por Monto a lo largo del Tiempo (Vintage)
# df_ratio_calidad_vintage (ya calculado)
if 'df_ratio_calidad_vintage' in globals():
    fig_ratio_vintage_line = px.line(df_ratio_calidad_vintage,
                                x='anio_otorgamiento',
                                y='ratio_incobrabilidad_por_monto_pct',
                                title='Evolución Tasa de Incobrabilidad por Monto (Vintage)',
                                markers=True,
                                labels={'ratio_incobrabilidad_por_monto_pct': 'Tasa de Incobrabilidad (%)', 'anio_otorgamiento': 'Año de Otorgamiento'})
    fig_ratio_vintage_line.update_layout(yaxis_ticksuffix='%')
    fig_ratio_vintage_line.show()

# 4. Mapa de Calor de Morosidad por Perfil (Edad y Ciudad)
# heatmap_data_morosidad (ya calculado y visualizado en Bloque 5)
if 'heatmap_data_morosidad' in globals() and not heatmap_data_morosidad.empty:
    plt.figure(figsize=(12, 8))
    sns.heatmap(heatmap_data_morosidad, annot=True, fmt=".1f", cmap="Reds", linewidths=.5)
    plt.title('Heatmap de Tasa de Morosidad (%) por Grupo de Edad y Ciudad', fontsize=15)
    plt.xlabel('Ciudad', fontsize=12)
    plt.ylabel('Grupo de Edad', fontsize=12)
    plt.show()
else:
    print("Datos para el heatmap de morosidad no disponibles o vacíos.")


# 5. Rentabilidad Promedio de Créditos Activos vs. Tasa de Morosidad por Ciudad
consulta_rent_moro_ciudad = """
SELECT
    cl.ciudad,
    COUNT(DISTINCT cr.credito_id) AS numero_creditos,
    ROUND(AVG(CASE WHEN cr.estado_credito = 'Activo' THEN cr.tasa_interes_anual ELSE NULL END), 2) AS rentabilidad_promedio_activos_pct,
    ROUND(SUM(CASE WHEN cr.estado_credito = 'Incobrable' THEN cr.monto_credito ELSE 0 END) * 100.0 / 
          SUM(cr.monto_credito), 2) AS tasa_morosidad_monto_pct
FROM creditos cr
JOIN clientes cl ON cr.cliente_id = cl.cliente_id
GROUP BY cl.ciudad
HAVING COUNT(DISTINCT cr.credito_id) > 0; -- Solo ciudades con créditos
"""
df_rent_moro_ciudad = ejecutar_consulta_sql(consulta_rent_moro_ciudad)
df_rent_moro_ciudad.fillna(0, inplace=True) # Para rentabilidad si no hay activos, o morosidad si no hay incobrables

print("\nRentabilidad Promedio de Créditos Activos vs. Tasa de Morosidad por Ciudad:")
print(df_rent_moro_ciudad)

if not df_rent_moro_ciudad.empty:
    fig_rent_moro_ciudad = px.scatter(df_rent_moro_ciudad,
                                     x='rentabilidad_promedio_activos_pct',
                                     y='tasa_morosidad_monto_pct',
                                     size='numero_creditos',
                                     color='ciudad',
                                     hover_name='ciudad',
                                     title='Rentabilidad vs. Morosidad de Créditos por Ciudad',
                                     labels={
                                         'rentabilidad_promedio_activos_pct': 'Rentabilidad Promedio Activos (%)',
                                         'tasa_morosidad_monto_pct': 'Tasa de Morosidad por Monto (%)'
                                     },
                                     size_max=25)
    fig_rent_moro_ciudad.update_layout(xaxis_ticksuffix='%', yaxis_ticksuffix='%')
    fig_rent_moro_ciudad.show()

#### 6.3. Dashboard de Actividad Transaccional y Flujos de Efectivo

Un vistazo a cómo se mueve el dinero, tanto a nivel agregado como por segmentos.

In [ ]:
print(f"\n--- Simulación de Dashboard de Actividad Transaccional y Flujos de Efectivo ---")

# 1. Evolución Mensual de Ingresos, Gastos e Ingreso Neto (Ya calculado)
# df_ingreso_neto_mensual
if 'df_ingreso_neto_mensual' in globals():
    # Usamos la figura ya creada en 5.4.1 que combina barras y línea con doble eje
    fig_ing_neto_mensual.show()


# 2. Distribución de Tipos de Transacción (Cantidad y Monto)
# df_tipo_transaccion (del Bloque 4)
if 'df_tipo_transaccion' in globals():
    fig_trans_cantidad_full = px.bar(df_tipo_transaccion.sort_values('cantidad', ascending=False),
                               x='tipo_transaccion',
                               y='cantidad',
                               color='tipo_transaccion',
                               title='Número Global de Transacciones por Tipo',
                               text_auto=True)
    fig_trans_cantidad_full.update_layout(xaxis_tickangle=-45)
    fig_trans_cantidad_full.show()

    # Monto neto
    df_tipo_transaccion_monto_neto = df_tipo_transaccion.copy()
    # Para el gráfico de cascada, es mejor tener los gastos como positivos y luego definirlos como decrementos
    df_tipo_transaccion_monto_neto['monto_abs_para_waterfall'] = df_tipo_transaccion_monto_neto['total_monto_neto'].abs()
    df_tipo_transaccion_monto_neto['efecto_waterfall'] = df_tipo_transaccion_monto_neto['total_monto_neto'].apply(lambda x: 'absolute' if x > 0 else 'relative')
    
    # Crear un gráfico de cascada (Waterfall) para el impacto de cada tipo de transacción en un "saldo inicial" teórico.
    # Esto es una visualización avanzada que Power BI maneja bien.
    # Necesitamos un punto de partida y un punto final.
    # Simplificaremos mostrando el impacto neto de cada tipo de transacción.
    
    # Mejor un bar chart que muestre ingresos y gastos por separado
    ingresos_tipos = df_tipo_transaccion[df_tipo_transaccion['total_monto_neto'] > 0].copy()
    gastos_tipos = df_tipo_transaccion[df_tipo_transaccion['total_monto_neto'] < 0].copy()
    gastos_tipos['total_monto_neto'] = gastos_tipos['total_monto_neto'].abs() # Convertir a positivo para el gráfico

    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    fig_ing_gas_tipos = make_subplots(rows=1, cols=2, subplot_titles=("Total Ingresado por Tipo (€)", "Total Gastado por Tipo (€)"))

    fig_ing_gas_tipos.add_trace(
        go.Bar(x=ingresos_tipos['tipo_transaccion'], y=ingresos_tipos['total_monto_neto'], name='Ingresos', marker_color='green'),
        row=1, col=1
    )
    fig_ing_gas_tipos.add_trace(
        go.Bar(x=gastos_tipos['tipo_transaccion'], y=gastos_tipos['total_monto_neto'], name='Gastos', marker_color='red'),
        row=1, col=2
    )
    fig_ing_gas_tipos.update_layout(title_text='Desglose de Ingresos y Gastos por Tipo de Transacción', showlegend=False)
    fig_ing_gas_tipos.update_xaxes(tickangle=-45)
    fig_ing_gas_tipos.show()


# 3. Segmentación de Clientes por Actividad Transaccional (RFM simplificado - Recencia, Frecuencia, Monto)
# Usaremos los datos de df_actividad_transaccional y añadiremos una pseudo-recencia
consulta_rfm_data = """
WITH UltimaTransaccionCliente AS (
    -- Fecha de la última transacción por cliente
    SELECT
        cu.cliente_id,
        MAX(t.fecha_transaccion) AS fecha_ultima_transaccion
    FROM transacciones t
    JOIN cuentas cu ON t.cuenta_id = cu.cuenta_id
    GROUP BY cu.cliente_id
),
ActividadGlobalCliente AS (
    -- Frecuencia (número de transacciones) y Monto (suma de montos absolutos de transacciones)
    SELECT
        cu.cliente_id,
        COUNT(t.transaccion_id) AS frecuencia,
        SUM(ABS(t.monto)) AS monetario_total -- Suma de todos los movimientos, independientemente del signo
    FROM transacciones t
    JOIN cuentas cu ON t.cuenta_id = cu.cuenta_id
    GROUP BY cu.cliente_id
)
SELECT
    cl.cliente_id,
    cl.nombre,
    (julianday('now') - julianday(utc.fecha_ultima_transaccion)) AS recencia_dias, -- Recencia en días
    agc.frecuencia,
    ROUND(agc.monetario_total, 2) AS monetario_total,
    NTILE(4) OVER (ORDER BY (julianday('now') - julianday(utc.fecha_ultima_transaccion)) ASC) AS r_score, -- Menos días = mejor score
    NTILE(4) OVER (ORDER BY agc.frecuencia DESC) AS f_score,
    NTILE(4) OVER (ORDER BY agc.monetario_total DESC) AS m_score
FROM clientes cl
JOIN UltimaTransaccionCliente utc ON cl.cliente_id = utc.cliente_id
JOIN ActividadGlobalCliente agc ON cl.cliente_id = agc.cliente_id
ORDER BY r_score, f_score, m_score; -- Prioriza mejores clientes
"""
df_rfm = ejecutar_consulta_sql(consulta_rfm_data)
df_rfm['recencia_dias'] = pd.to_numeric(df_rfm['recencia_dias'], errors='coerce')
df_rfm.dropna(subset=['recencia_dias'], inplace=True) # Eliminar si hay algún nulo por cálculo de fecha
df_rfm['recencia_dias'] = df_rfm['recencia_dias'].astype(int)


print("\nSegmentación RFM de Clientes (muestra):")
print(df_rfm.head(10))

# Combinar scores RFM en un solo segmento
df_rfm['segmento_rfm'] = df_rfm['r_score'].astype(str) + df_rfm['f_score'].astype(str) + df_rfm['m_score'].astype(str)

# Contar clientes por segmento RFM combinado
rfm_summary = df_rfm.groupby('segmento_rfm').agg(
    numero_clientes = ('cliente_id', 'count'),
    recencia_media_dias = ('recencia_dias', 'mean'),
    frecuencia_media = ('frecuencia', 'mean'),
    monetario_medio = ('monetario_total', 'mean')
).reset_index().sort_values('numero_clientes', ascending=False)

print("\nResumen de Segmentos RFM:")
print(rfm_summary.head())

# Visualización de los segmentos RFM más poblados
fig_rfm_segments = px.bar(rfm_summary.head(10), 
                          x='segmento_rfm', 
                          y='numero_clientes',
                          color='segmento_rfm',
                          title='Top 10 Segmentos RFM por Número de Clientes',
                          text_auto=True)
fig_rfm_segments.show()

# Un treemap para visualizar la distribución del valor monetario total por segmento RFM
# Para esto, necesitamos el monetario total por segmento, no solo el promedio
rfm_monetary_sum = df_rfm.groupby('segmento_rfm')['monetario_total'].sum().reset_index()
# Para que el treemap sea legible, podríamos agrupar por R_score y luego por F_score
df_rfm['rfm_group_label'] = 'R' + df_rfm['r_score'].astype(str) + '-F' + df_rfm['f_score'].astype(str) #+ '-M' + df_rfm['m_score'].astype(str)
# Path para treemap: r_score -> f_score -> m_score
# Para simplificar, podemos hacer un treemap de los segmentos 'RFM generales' (ej. Campeones, Leales, etc.)
# Esta definición de segmentos RFM generales es subjetiva y puede variar.
def asignar_segmento_rfm_general(row):
    if row['r_score'] == 1 and row['f_score'] == 1 and row['m_score'] == 1: return 'Campeones (111)'
    if row['r_score'] <= 2 and row['f_score'] <= 2 : return 'Clientes Leales (R<=2, F<=2)'
    if row['m_score'] == 1 : return 'Grandes Compradores (M=1)'
    if row['r_score'] == 4 and row['f_score'] == 4 : return 'En Riesgo / Perdidos (R=4, F=4)'
    if row['f_score'] == 1 : return 'Compradores Frecuentes (F=1)'
    if row['r_score'] == 1 : return 'Compradores Recientes (R=1)'
    return 'Otros'

df_rfm['segmento_general'] = df_rfm.apply(asignar_segmento_rfm_general, axis=1)
rfm_general_summary = df_rfm.groupby('segmento_general').agg(
    numero_clientes = ('cliente_id', 'count'),
    monetario_total_segmento = ('monetario_total', 'sum')
).reset_index()

fig_rfm_treemap = px.treemap(rfm_general_summary,
                             path=[px.Constant("Todos los Clientes"), 'segmento_general'],
                             values='monetario_total_segmento',
                             color='monetario_total_segmento',
                             color_continuous_scale='Blues',
                             title='Distribución del Valor Monetario por Segmento RFM General')
fig_rfm_treemap.show()

# BLOQUE 7: Exportación de Resultados y Conclusiones Accionables

#### 7.1. Exportación de Resultados Clave a Excel

Guardaremos los DataFrames más relevantes generados durante nuestro análisis en un archivo Excel, con cada DataFrame en una hoja separada. Esto facilita la revisión y el uso posterior de los datos.

In [ ]:
# Listado de DataFrames clave que hemos generado y queremos exportar:

# Datos base 
# - clientes
# - cuentas
# - transacciones
# - creditos

# Resultados del análisis
# - df_total_cuentas (Resumen de tipos de cuenta)
# - df_clientes_valor (Segmentación de clientes por valor - cuartiles)
# - df_antiguedad_valor (Relación antigüedad, saldo, créditos)
# - df_morosidad_perfil (Morosidad por edad y ciudad)
# - df_vintage_creditos (Análisis vintage de créditos - estado por año)
# - df_vintage_porcentaje (Análisis vintage de créditos - porcentaje por estado)
# - df_actividad_transaccional (Frecuencia y monto promedio de transacciones por cliente)
# - df_patrones_gasto (Principales gastos mensuales por cliente - muestra)
# - df_gasto_agregado_tipo (Total gastado por tipo de transacción de salida)
# - df_ingreso_neto_mensual (Ingreso neto mensual y acumulado)
# - df_ratio_calidad_activos (Ratio de calidad de activos - general)
# - df_ratio_calidad_vintage (Ratio de incobrabilidad por monto - vintage)
# - df_rent_moro_ciudad (Rentabilidad vs Morosidad por ciudad)
# - df_rfm (Datos base para RFM con scores)
# - rfm_summary (Resumen de segmentos RFM combinados)
# - rfm_general_summary (Resumen de segmentos RFM generales con valor monetario)

# Ruta del archivo Excel
ruta_excel_completo = "analisis_financiero_banco_detallado_v2.xlsx"

print(f"Iniciando exportación a Excel: {ruta_excel_completo}...")

with pd.ExcelWriter(ruta_excel_completo, engine='xlsxwriter') as writer:
    # Hoja de Resumen/Índice (Opcional, pero útil)
    resumen_proyecto = pd.DataFrame({
        'Sección': [
            'Datos Base - Clientes', 'Datos Base - Cuentas', 'Datos Base - Transacciones', 'Datos Base - Créditos',
            'Análisis - Resumen Tipos Cuenta', 'Análisis - Clientes por Valor (Saldo)', 'Análisis - Antigüedad Clientes',
            'Análisis - Morosidad por Perfil', 'Análisis - Créditos Vintage (Estado)', 'Análisis - Créditos Vintage (% Estado)',
            'Análisis - Actividad Transaccional', 'Análisis - Patrones Gasto (Muestra)', 'Análisis - Gasto Agregado por Tipo',
            'KPIs - Ingreso Neto Mensual', 'KPIs - Ratio Calidad Activos (General)', 'KPIs - Ratio Incobrabilidad (Vintage)',
            'Análisis - Rentabilidad vs Morosidad (Ciudad)', 'Segmentación - RFM Detallado', 
            'Segmentación - RFM Resumen (Combinado)', 'Segmentación - RFM Resumen (General)'
        ],
        'Descripción': [
            'Información demográfica y de registro de los clientes.',
            'Detalles de las cuentas bancarias, tipos y saldos.',
            'Registro de todas las transacciones financieras.',
            'Información sobre los créditos otorgados a los clientes.',
            'Número de cuentas, saldo promedio y total por tipo de cuenta.',
            'Clientes segmentados por cuartiles según su saldo total.',
            'Análisis de la antigüedad del cliente y su relación con saldos y créditos.',
            'Tasa de morosidad de créditos desglosada por grupo de edad y ciudad del cliente.',
            'Estado actual de los créditos agrupados por su año de otorgamiento (cantidad y monto).',
            'Distribución porcentual del estado de los créditos por año de otorgamiento.',
            'Frecuencia y monto promedio de las transacciones por cliente.',
            'Muestra de los principales tipos de gasto mensuales para algunos clientes.',
            'Monto total gastado a nivel general del banco, desglosado por tipo de transacción de salida.',
            'Evolución mensual y acumulada del ingreso neto basado en transacciones.',
            'Indicadores clave de la calidad de la cartera de créditos (por número y por monto).',
            'Evolución de la tasa de incobrabilidad (por monto) de los créditos según su año de otorgamiento.',
            'Comparativa de la rentabilidad promedio de créditos activos y la tasa de morosidad por ciudad.',
            'Datos detallados de la segmentación RFM (Recencia, Frecuencia, Monetario) para cada cliente.',
            'Resumen de los segmentos RFM combinados (ej. "111", "112") y sus métricas promedio.',
            'Resumen de los segmentos RFM generales (ej. "Campeones") con número de clientes y valor monetario total.'
        ],
        'DataFrame Original': [
            'clientes', 'cuentas', 'transacciones', 'creditos',
            'df_total_cuentas', 'df_clientes_valor', 'df_antiguedad_valor',
            'df_morosidad_perfil', 'df_vintage_creditos', 'df_vintage_porcentaje',
            'df_actividad_transaccional', 'df_patrones_gasto', 'df_gasto_agregado_tipo',
            'df_ingreso_neto_mensual', 'df_ratio_calidad_activos', 'df_ratio_calidad_vintage',
            'df_rent_moro_ciudad', 'df_rfm', 
            'rfm_summary', 'rfm_general_summary'
        ]
    })
    resumen_proyecto.to_excel(writer, sheet_name="Indice_Contenidos", index=False)
    
    # Exportar cada DataFrame a una hoja diferente
    # Datos base
    if 'clientes' in globals(): clientes.to_excel(writer, sheet_name="Clientes", index=False)
    if 'cuentas' in globals(): cuentas.to_excel(writer, sheet_name="Cuentas", index=False)
    if 'transacciones' in globals(): transacciones.to_excel(writer, sheet_name="Transacciones", index=False)
    if 'creditos' in globals(): creditos.to_excel(writer, sheet_name="Creditos", index=False)
    
    # Resultados del análisis y KPIs
    if 'df_total_cuentas' in globals(): df_total_cuentas.to_excel(writer, sheet_name="Resumen_Tipos_Cuenta", index=False)
    if 'df_clientes_valor' in globals(): df_clientes_valor.to_excel(writer, sheet_name="Clientes_por_Valor", index=False)
    if 'df_antiguedad_valor' in globals(): df_antiguedad_valor.to_excel(writer, sheet_name="Antiguedad_Saldo_Creditos", index=False)
    if 'df_morosidad_perfil' in globals(): df_morosidad_perfil.to_excel(writer, sheet_name="Morosidad_Perfil_Cliente", index=False)
    if 'df_vintage_creditos' in globals(): df_vintage_creditos.to_excel(writer, sheet_name="Creditos_Vintage_Abs", index=False)
    if 'df_vintage_porcentaje' in globals(): df_vintage_porcentaje.to_excel(writer, sheet_name="Creditos_Vintage_Pct", index=False)
    if 'df_actividad_transaccional' in globals(): df_actividad_transaccional.to_excel(writer, sheet_name="Actividad_Transacc_Cliente", index=False)
    if 'df_patrones_gasto' in globals(): df_patrones_gasto.to_excel(writer, sheet_name="Patrones_Gasto_Muestra", index=False)
    if 'df_gasto_agregado_tipo' in globals(): df_gasto_agregado_tipo.to_excel(writer, sheet_name="Gasto_Agregado_Tipo", index=False)
    if 'df_ingreso_neto_mensual' in globals(): df_ingreso_neto_mensual.to_excel(writer, sheet_name="Ingreso_Neto_Mensual", index=False)
    if 'df_ratio_calidad_activos' in globals(): df_ratio_calidad_activos.to_excel(writer, sheet_name="Ratio_Calidad_Activos", index=False)
    if 'df_ratio_calidad_vintage' in globals(): df_ratio_calidad_vintage.to_excel(writer, sheet_name="Ratio_Incobrabilidad_Vintage", index=False)
    if 'df_rent_moro_ciudad' in globals(): df_rent_moro_ciudad.to_excel(writer, sheet_name="Rentab_Mora_Ciudad", index=False)
    if 'df_rfm' in globals(): df_rfm.to_excel(writer, sheet_name="RFM_Detallado", index=False)
    if 'rfm_summary' in globals(): rfm_summary.to_excel(writer, sheet_name="RFM_Resumen_Combinado", index=False)
    if 'rfm_general_summary' in globals(): rfm_general_summary.to_excel(writer, sheet_name="RFM_Resumen_General", index=False)

    print(f"✅ Archivo Excel '{ruta_excel_completo}' generado exitosamente con múltiples hojas.")